In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard library imports.
import itertools
import json

# Related third party imports.
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots

# Local application/library specific imports.
import src.visualization.visualize as visualize


import os
os.environ["PATH"] += os.pathsep + '/home/ubuntu/texlive/2022/bin/x86_64-linux'

plt.style.use(['science', 'ieee', 'no-latex'])
# plt.rcParams['figure.dpi'] = 150

pd.options.display.float_format = '{:,.2f}'.format

## Save json to csv

In [ ]:
network = '25_italy'
simulation_date = '2021-11-08'
placement_type = 'heu-hco'

df = pd.read_json(f"../results/{network}/{simulation_date}-{network}-{placement_type}.json")
df.latency_factor = df.latency_factor.round(1)

In [ ]:
df.to_csv(f"../results/{network}/2021-11-04-{network}-ilp-hco.csv", index=False)

In [ ]:
df.query(" vSDN_size == 2 ")

## Analyse Data

In [ ]:
network = '37_cost'
simulation_date = '2021-11-12'

placement_type = 'heu-hco'
dfh = pd.read_json(f"../results/{network}/static/{simulation_date}-{network}-{placement_type}.json")
dfh.latency_factor = dfh.latency_factor.round(1)

placement_type = 'ilp-hco'
dfi = pd.read_json(f"../results/{network}/static/{simulation_date}-{network}-{placement_type}.json")
dfi.latency_factor = dfi.latency_factor.round(1)

placement_type = 'ilp-acc'
dfa = pd.read_json(f"../results/{network}/static/{simulation_date}-{network}-{placement_type}.json")
dfa.latency_factor = dfa.latency_factor.round(1)

# placement_type = 'ilp-opt'
# dfo = pd.read_json(f"../results/{network}/static/{simulation_date}-{network}-{placement_type}.json")
# dfo.latency_factor = dfo.latency_factor.round(1)

In [ ]:
for df in [dfh, dfi, dfa]:
    print(df.query("latency_factor == 0.4 & vSDN_size == 8")[['acceptance_ratio','hp_runtime']].mean())

In [ ]:
dfh.query("latency_factor == 0.5 & vSDN_size == 6")[['acceptance_ratio','hp_runtime','h_list']]

In [ ]:
dfi.query("latency_factor == 0.5 & vSDN_size == 6")[['acceptance_ratio','hp_runtime','h_list']]

In [ ]:
dfa.query("latency_factor == 0.5 & vSDN_size == 6")[['acceptance_ratio','hp_runtime','h_list']]

In [ ]:
dfo.query("latency_factor == 0.5 & vSDN_size == 6")[['acceptance_ratio','hp_runtime','h_list']]

## Save data for latex/pgfplots

In [ ]:
network = '37_cost'
simulation_date = '2021-11-12'
placement_type = 'heu-hco'

df = pd.read_json(f"../results/{network}/static/{simulation_date}-{network}-{placement_type}.json")
df.latency_factor = df.latency_factor.round(1)
df.drop(labels=['h_list'], axis=1, inplace=True)
df = df.groupby(by=['latency_factor','vSDN_size']).mean()

In [ ]:
df.to_csv(f"../results/{network}/static/{simulation_date}-{network}-{placement_type}-mean-pgf.csv")

# Hypervisor Count Effect

In [ ]:
json_list = [
    '../results/25_italy/static/tmp/2023-04-03-21-43-35/simulation-group-results.json',
    '../results/25_italy/static/tmp/2023-04-16-14-16-49/simulation-group-results.json',
    '../results/25_italy/static/tmp/2023-04-16-17-41-00/simulation-group-results.json',
    '../results/25_italy/static/tmp/2023-04-17-10-59-37/simulation-group-results.json',
]
df = pd.concat([pd.read_json(json_file) for json_file in json_list])

In [ ]:
df.n_hypervisors.value_counts()

In [ ]:
fig, ax = plt.subplots()

for n_hypervisors, group in df.groupby('n_hypervisors'):
    if n_hypervisors not in [6,8,10,12,14]:
        continue
    tmp = group.groupby('vSDN_size').mean()
    ax.plot(tmp.index, tmp.vSDN_acceptance_ratio, label=n_hypervisors)
    # if n_hypervisors == 5:
    #     break

ax.grid(True)
ax.legend([6,8,10,12,"14 (max)"], title='No. hypervisors', framealpha=1, frameon=True)
ax.set_xlabel('vSDN size')
ax.set_ylabel('Acceptance ratio')

plt.savefig('../reports/figures/2023-RNDM/italy_25_hypervisor_count_effect.pdf')

# Heuristic evaluation

In [ ]:
dfi = pd.read_json('../results/26_usa/static/tmp/2023-04-16-11-00-13/simulation-group-results.json')

In [ ]:
json_list = [
    '../results/26_usa/static/tmp/2023-04-15-16-41-05/simulation-group-results.json', # 1-100
    '../results/26_usa/static/tmp/2023-04-16-10-54-03/simulation-group-results.json', # 200, 400
    '../results/26_usa/static/tmp/2023-04-19-18-38-31/simulation-group-results.json', # 0.3, 0.5
    # '../results/25_italy/static/tmp/2023-04-19-18-26-47/simulation-group-results.json', # 1-50
]

dfh = pd.concat([pd.read_json(json_file) for json_file in json_list])
dfh.heuristic_randomness = dfh.heuristic_randomness.round(2)
dfh = dfh.astype({'repeat': 'category', 'heuristic_randomness': 'category'})
# dfh['repeat'] = dfh['repeat'].astype('category')
# dfh['heuristic_randomness'] = dfh['heuristic_randomness'].astype('category')

In [ ]:
dfh['heuristic_randomness'].value_counts()

In [ ]:
print(dfh['repeat'].unique())
print(dfh['heuristic_randomness'].unique())

In [ ]:
plt.figure(figsize=visualize.set_size(600, subplots=(1, dfh['repeat'].nunique())))
plt.subplots_adjust(wspace=0.25)

# plt.suptitle("USA 26 - Static vSDN placement - Heuristic", fontsize=20)

for i, (repeat, group1) in enumerate(dfh.groupby(by=['repeat'])):
    ax = plt.subplot(1, dfh['repeat'].nunique(), i+1)
    ax.set_title(f"No. repeat: {repeat}")
    for heuristic_randomness, group2 in group1.groupby('heuristic_randomness'):
        tmp = group2.groupby('vSDN_size').mean()
        ax.plot(tmp.index, tmp.vSDN_acceptance_ratio, label=f'{heuristic_randomness:.1f}')
        tmpi = dfi.groupby('vSDN_size').mean()
        ax.plot(tmpi.index, tmpi.vSDN_acceptance_ratio, label='ILP', linestyle='--', color='black')
    ax.grid()
    ax.set_xlabel('vSDN size')
    if i == 0:
        ax.set_ylabel('Acceptance ratio')
    else:
        ax.set_ylabel('')
    if i == 0:
        ax.legend([])
        # ax.legend(title='Randomness', framealpha=1, frameon=True, fontsize=8)
    else:
        ax.legend([])


# plt.savefig('../reports/figures/2023-RNDM/usa_26_heuristic_randomness.pdf')

In [ ]:
dfh_total_count = dfh.groupby(by=['repeat', 'heuristic_randomness'], as_index=False).agg({'vSDN_accepted_count': ['sum']})
dfh_total_count.columns = [col_name[0] for col_name in dfh_total_count.columns]
dfh_total_count_pivot = pd.pivot_table(dfh_total_count, values="vSDN_accepted_count",index=["repeat"], columns=["heuristic_randomness"], fill_value=0)
dfh_total_count_pivot = dfh_total_count_pivot / dfi['vSDN_accepted_count'].sum()
dfh_total_count_pivot

In [ ]:
print(dfh_total_count_pivot.to_latex())

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(dfh_total_count_pivot, linewidth=.5, cmap="YlGnBu", ax=ax)

plt.yticks(rotation=0)
plt.xticks(rotation=0)
ax.grid(visible=False)

# plt.savefig('../reports/figures/2023-RNDM/usa_26_heuristic_randomness_heatmap.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.lineplot(data=dfh, x='vSDN_size', y='vSDN_acceptance_ratio', hue='simulation_round', ax=ax)
sns.lineplot(data=dfh, x='vSDN_size', y='vSDN_acceptance_ratio', ax=ax)
ax.grid()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dfh.groupby(by=['vSDN_size']).mean().plot(y='vSDN_acceptance_ratio', ax=ax, label='heu_improved')
ax.grid()

In [ ]:
dfh.active_hypervisors.apply(tuple).value_counts()

In [ ]:
dfi = pd.read_json('../results/25_italy/static/tmp/2023-03-27-13-00-06/simulation-group-results.json')

In [ ]:
dfi.active_hypervisors.apply(tuple).value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dfh.groupby(by=['vSDN_size']).mean().plot(y='vSDN_acceptance_ratio', ax=ax, label='heu_improved')
dfi.groupby(by=['vSDN_size']).mean().plot(y='vSDN_acceptance_ratio', ax=ax, label='ilp_acc')
ax.grid()

# Hypervisor capacity

In [ ]:
json_path = '../results/26_usa/static/tmp/2023-04-07-12-13-59/simulation-group-results.json'
df = pd.read_json(json_path)
df = df.astype({'hypervisor_capacity': 'category'})

In [ ]:
df.n_hypervisors.value_counts()

In [ ]:
fig, ax = plt.subplots()

for hypervisor_capacity, hc_group in df.groupby('hypervisor_capacity'):
    for n_hypervisors, nh_group in hc_group.groupby('n_hypervisors'):
        if hypervisor_capacity not in [12,14,15,16,22] or n_hypervisors != 4:
            continue
        tmp = nh_group.groupby('vSDN_size').mean()
        ax.plot(tmp.index, tmp.vSDN_acceptance_ratio, label=f"{hypervisor_capacity} / {tmp.n_hypervisors.max():.0f}", alpha=1)

ax.grid(True)
ax.legend(title='Hypervisor\ncapacity/count', framealpha=1, frameon=True)
ax.set_xlabel('vSDN size')
ax.set_ylabel('Acceptance ratio')

# plt.savefig('../reports/figures/usa_26_hypervisor_capacity_effect.pdf')

In [ ]:
sns.lineplot(
    data=dfh,
    x='vSDN_size', y='vSDN_acceptance_ratio', hue='hypervisor_capacity'
)

# Controller capacity

In [ ]:
dfh = pd.read_json('../results/25_italy/static/2023-02-10-12-44-28/simulation-group-results.json')
dfh = dfh.astype({'controller_capacity': 'category'})

In [ ]:
dfh.controller_capacity.value_counts()

In [ ]:
for cc, group in dfh.groupby(by='controller_capacity'):
    print(cc, group.h_list.iloc[0])

In [ ]:
sns.lineplot(
    data=dfh,
    x='vSDN_size', y='acceptance_ratio', hue='controller_capacity'
)

# Flexibility

In [ ]:
dfh = pd.read_json('../results/25_italy/static/tmp/2023-02-20-13-11-01/simulation-group-results.json')
# dfh = dfh.astype({'acceptance_ratio_ilp':int})

In [ ]:
sns.lineplot(
    data=dfh,
    x='vSDN_size', y='vSDN_acceptance_ratio', hue='simulation_round'
)

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))

for hd in [0,1,2,3]:
    for fw in [2]:
        dfh[(3*hd+fw)*115:(3*hd+fw+1)*115].groupby(by='vSDN_size').mean()\
            .plot(y='acceptance_ratio', ax=ax, label=f"{hd}-{fw}")

# Multiple Objectives

In [ ]:
json_list = [
    # '../results/25_italy/static/tmp/2023-04-15-16-27-39/simulation-group-results.json',
    # '../results/25_italy/static/tmp/2023-04-15-16-27-54/simulation-group-results.json',
    # '../results/25_italy/static/tmp/2023-04-15-16-28-07/simulation-group-results.json',
    # '../results/25_italy/static/tmp/2023-04-15-16-28-19/simulation-group-results.json',
    # '../results/25_italy/static/tmp/2023-04-15-16-28-32/simulation-group-results.json',
    # '../results/25_italy/static/tmp/2023-04-15-16-29-06/simulation-group-results.json',
    # '../results/25_italy/static/tmp/2023-04-15-16-30-11/simulation-group-results.json',
    # '../results/25_italy/static/tmp/2023-04-15-16-30-37/simulation-group-results.json',
    '../results/25_italy/static/tmp/2023-04-16-13-33-14/simulation-group-results.json',
]

df = pd.concat([pd.read_json(json_path) for json_path in json_list])
df['hp_objectives'] = df['hp_objectives'].apply(lambda x: tuple(x)).astype('category')
df['hp_objectives'].value_counts()

In [ ]:
df_mo = df[['hp_objectives', 'controller_load_request_max_ilp', 'controller_load_switch_max_ilp', 'hypervisor_load_max_ilp', 'vSDN_accepted_count_ilp']].groupby(by='hp_objectives').mean()
# df_mo = df_mo / df_mo.max()
df_mo['controller_load_request_max_ilp'] = df_mo['controller_load_request_max_ilp'] / df_mo['vSDN_accepted_count_ilp'].max()
df_mo['hypervisor_load_max_ilp'] = df_mo['hypervisor_load_max_ilp'] / df['n_nodes'].max()
df_mo

In [ ]:
print(df_mo.to_latex())

# Capacity

In [ ]:
df = pd.read_json('../results/25_italy/static/tmp/2023-04-15-16-32-42/simulation-group-results.json')
# for col in ['controller_capacity', 'hypervisor_capacity']:
#     df[col] = df[col].astype('category')

In [ ]:
df['hypervisor_capacity'].value_counts()

In [ ]:
df.groupby(by=['controller_capacity']).agg({'vSDN_accepted_count': 'mean', 'vSDN_accepted_count_ilp': 'mean'})

In [ ]:
cols = ['controller_capacity', 'hypervisor_capacity', 'vSDN_accepted_count']

dfp = df.groupby(by=cols[:2], as_index=False).agg({cols[2]: ['sum']})
# dfp.columns = [col_name[0] for col_name in dfp.columns]
# dfp = pd.pivot_table(dfp, values=cols[2],index=[cols[0]], columns=[cols[1]], fill_value=0)
dfp